In [ ]:
#default_exp convolutions_pooling_04

In [ ]:
#export
from ModernArchitecturesFromScratch.basic_operations_01 import *
from ModernArchitecturesFromScratch.fully_connected_network_02 import *
from ModernArchitecturesFromScratch.model_training_03 import *
from nbdev.showdoc import *

# Convolutions and Pooling
> Implementing forwards and backwards passes for convolution and pooling layers as well as support for padding and stride

In [ ]:
#hide
train, valid = get_datasets()
m, o, lf = get_model(0.1)
fit(3, m, o, lf, train, valid)

Epoch 1, Accuracy: 0.9368033409118652, Loss: 0.19594129920005798
Epoch 2, Accuracy: 0.9617834687232971, Loss: 0.13096937537193298
Epoch 3, Accuracy: 0.962082028388977, Loss: 0.13402025401592255


## Tensor Shaping Modules

In [ ]:
#export
class Reshape(Module):
    "Module to reshape input tensor into tensor of (bs, `channels`, `size1`, `size2`)"
    def __init__(self, channels, size1, size2):
        super().__init__()
        self.size1 = size1
        self.size2 = size2
        self.channels = channels
    
    def forward(self, xb): return xb.view(-1, self.channels, self.size1, self.size2)
    
    def bwd(self, out, inp): 
        inp.g = out.g.reshape(-1, self.channels*self.size1*self.size2)
    
    def __repr__(self): return f'Reshape({self.channels}, {self.size1}, {self.size2})'

```python
class Reshape(Module):
    "Module to reshape input tensor into tensor of (bs, `channels`, `size1`, `size2`)"
    def __init__(self, channels, size1, size2):
        super().__init__()
        self.size1 = size1
        self.size2 = size2
        self.channels = channels
    
    def forward(self, xb): return xb.view(-1, self.channels, self.size1, self.size2)
    
    def bwd(self, out, inp): 
        inp.g = out.g.reshape(-1, self.channels*self.size1*self.size2)
    
    def __repr__(self): return f'Reshape({self.channels}, {self.size1}, {self.size2})'```

In [ ]:
#export
class Flatten(Module):
    "Module to flatten tensor input into shape (bs, rest)"
    def __init__(self):
        super().__init__()
    
    def forward(self, xb): 
        self.size1 = xb.shape[2]
        self.size2 = xb.shape[3]
        self.channels = xb.shape[1]
        return xb.view(xb.shape[0],-1)
    
    def bwd(self, out, inp): inp.g = out.g.view(-1, self.channels, self.size1, self.size2)
        
    def __repr__(self): return f'Flatten()'

```python
class Flatten(Module):
    "Module to flatten tensor input into shape (bs, rest)"
    def __init__(self):
        super().__init__()
    
    def forward(self, xb): 
        self.size1 = xb.shape[2]
        self.size2 = xb.shape[3]
        self.channels = xb.shape[1]
        return xb.view(xb.shape[0],-1)
    
    def bwd(self, out, inp): inp.g = out.g.view(-1, self.channels, self.size1, self.size2)
        
    def __repr__(self): return f'Flatten()'```

## Initialization

In [ ]:
#export

def get_fan(dim1, dim2, dim3, dim4, fan_out):
    "Get the appropriate fan value based on the receptive field size and number of activations in the previous layer"
    if dim3 == None and dim4 == None: return dim1
    else:
        rec_fs = dim3*dim4
        return dim2*rec_fs if fan_out else dim1*rec_fs
    
def get_gain(leak): 
    "Get proper initialization gain factor based on the leak amount of the next layer. Leak of 1 if no ReLU after"
    return math.sqrt(2.0 / (1 + leak**2))
    
def get_weight(dim1, dim2, dim3 = None, dim4 = None, leak=1., fan_out=False):
    "Improved Kaiming initialization to handle convolutional layers. Uses `get_gain` and `get_fan` to get appropriate values"
    fan = get_fan(dim1, dim2, dim3, dim4, fan_out)
    gain = get_gain(leak)
    std = gain / math.sqrt(fan)
    if dim3 == None and dim4 == None: return torch.randn(dim1, dim2) * std
    else: return torch.randn(dim1, dim2, dim3, dim4) * std

Please see: https://arxiv.org/abs/1502.01852 for more details and explanation on Kaiming initialisation. The idea is to regularize the model by keeping the mean and standard deviation close to 0 and 1 respectively.

In [ ]:
import torch.nn as nn
from torch.nn import init
test_conv = nn.Conv2d(1, 8, 5)
xt, _, _, _ = get_mnist()
xt = xt.view(-1, 1, 28, 28)
test_conv.weight.shape

torch.Size([8, 1, 5, 5])

Without proper initialization

In [ ]:
get_stats(test_conv(xt))

Mean: -0.010123089887201786
Std: 0.6414140462875366


Using my own regularization (mean is half due to the ReLU activation)

In [ ]:
test_conv.weight = nn.Parameter(get_weight(8, 1, 5, 5, 0, True)) 
get_stats(relu(test_conv(xt)))

Mean: 0.5077283382415771
Std: 0.7732385993003845


Pytorch regularization

In [ ]:
init.kaiming_normal_(test_conv.weight, a=0.)
get_stats(relu((test_conv(xt))))

Mean: 0.5470803380012512
Std: 1.1031794548034668


## Padding

In [ ]:
#export
class Padding():
    "Adds padding around an image `size` pixels wide."
    def __init__(self, size=1, mode="constant", value=0):
        self.size, self.mode, self.value = size, mode, value
    
    def __call__(self, tensor):
        if self.mode == "constant": return torch.nn.functional.pad(tensor, [self.size,self.size,self.size,self.size], value=self.value)
    
    def __repr__(self): return f'Padding (Mode: {self.mode}, Size: {self.size}, Value: {self.value})'

Can input any of the given modes:<br>
`Constant`: Adds a constant pixel of value `value` around the image<br>
`Reflection`: Repeats the outer most pixel value of the actual image


## Forward

In [ ]:
#export
def convolve(weight, filts, filts_bias, stride=1,padding=None):
    "Performs a convolution on `weight` using the given `filts` and bias `filts_bias`. Can specify a `stride for the convolution or `padding` for the main image."
    n_filt, depth_f, f_w, f_h = filts.shape
    bs, depth_im, im_w, im_h = weight.shape

    if padding is not None: 
        weight = padding(weight)
        p_s = padding.size
    else: p_s = 0
        
    _,_,p_w, p_h = weight.shape
    
    assert depth_f == depth_im

    final = torch.zeros(bs, n_filt, int((im_w + 2*p_s - f_w)/stride)+1, int((im_h + 2*p_s - f_h)/stride)+1)
    for j in range(0, p_w-f_h+1, stride): #vertical passes
        for k in range(0, p_h-f_w+1, stride): #horizontal passes
            final[:,:,j//stride,k//stride] = (weight[:,:,j:j+f_h,k:k+f_w].unsqueeze(1)*filts).sum(dim=-1).sum(dim=-1).sum(dim=-1) + filts_bias.unsqueeze(0)
                
    return final    

```python
def convolve(weight, filts, filts_bias, stride=1,padding=None):
    n_filt, depth_f, f_w, f_h = filts.shape
    bs, depth_im, im_w, im_h = weight.shape
    
    if padding is not None: 
        weight = padding(weight)
        p_s = padding.size
    else: p_s = 0
        
    _,_,p_w, p_h = weight.shape
    
    assert depth_f == depth_im

    final = torch.zeros(bs, n_filt, int((im_w + 2*p_s - f_w)/stride)+1, int((im_h + 2*p_s - f_h)/stride)+1)
    for j in range(0, p_w-f_h+1, stride): #vertical passes
        for k in range(0, p_h-f_w+1, stride): #horizontal passes
            final[:,:,j//stride,k//stride] = (weight[:,:,j:j+f_h,k:k+f_w].unsqueeze(1)*filts).sum(dim=-1).sum(dim=-1).sum(dim=-1) + filts_bias.unsqueeze(0)
                
    return final 
```

Testing against PyTorch's convolution:

In [ ]:
weight = torch.randn(10, 16, 28, 28)
bs = weight.shape[0]
im_h = im_w = weight.shape[2]
w_dim = weight.shape[1]
weight.shape

torch.Size([10, 16, 28, 28])

In [ ]:
stride = 3
pad_amount = 2

In [ ]:
f_dim = 4
f_w = 6
f_h = f_w
test_conv = nn.Conv2d(w_dim, f_dim, f_w, padding=pad_amount, stride=stride)
f = test_conv.weight
test_conv.weight.shape

torch.Size([4, 16, 6, 6])

In [ ]:
b = test_conv.bias
b.shape

torch.Size([4])

In [ ]:
pt_res = test_conv(weight)
pt_res.shape

torch.Size([10, 4, 9, 9])

In [ ]:
pad = Padding(size=pad_amount)
pad

Padding (Mode: constant, Size: 2, Value: 0)

In [ ]:
res = convolve(weight, f, b, stride, pad)

In [ ]:
test_near(res, pt_res)

good


## Backward

In [ ]:
#export
def conv_back(out,inp,weight,bias,stride=1,padding=None):
    "Performs a backward pass to get the gradient of the output, `out`, with respect to the `inp`, `weight` of filters and `bias`."
    dZ = out.g

    (A_prev, W, b, stride) = inp, weight.d, bias.d, stride
    
    # Retrieve dimensions from A_prev's shape
    (m, n_C_prev, n_W_prev, n_H_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape
    (n_C, n_C_prev, f, f) = W.shape
    
    # Retrieve dimensions from dZ's shape
    (m, n_C, n_W, n_H) = dZ.shape
    
    # Initialize dA_prev, dW, db with the correct shapes
    dA_prev = torch.zeros((m, n_C_prev, n_W_prev, n_H_prev))                           
    dW = torch.zeros((n_C, n_C_prev, f, f))
    db = torch.zeros((n_C, 1, 1, 1))

    # Pad A_prev and dA_prev
    if padding is not None: 
        A_prev = padding(A_prev)
        dA_prev = padding(dA_prev)
        
    for h in range(n_H): # loop over vertical axis of the output volume
        for w in range(n_W):               # loop over horizontal axis of the output volume
                
                # Find the corners of the current "slice"
                vert_start = h*stride
                vert_end = vert_start + f
                horiz_start = w*stride
                horiz_end = horiz_start + f
                    
                # Use the corners to define the slice from a_prev_pad
                a_slice = A_prev[:, :, horiz_start:horiz_end, vert_start:vert_end]
                
                ezdz = dZ[:, :, w, h].unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)

                # Update gradients for the filter, bias and input
                dA_prev[:, :, horiz_start:horiz_end, vert_start:vert_end] += (W * ezdz).sum(dim=1)
                dW += (a_slice.unsqueeze(1)*ezdz).sum(dim=0)
                db += dZ[:, :, w, h].sum(dim=0).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
                
    if padding is not None: dA_prev = dA_prev[:, :, padding.size:-padding.size, padding.size:-padding.size]
    
    weight.update(dW)
    bias.update(db.view(-1))
    inp.g = dA_prev

```python
def conv_back(out,inp,weight,bias,stride=1,padding=None):
    "Performs a backward pass to get the gradient of the output, `out`, with respect to the `inp`, `weight` of filters and `bias`."
    dZ = out.g

    (A_prev, W, b, stride) = inp, weight.d, bias.d, stride
    
    # Retrieve dimensions from A_prev's shape
    (m, n_C_prev, n_W_prev, n_H_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape
    (n_C, n_C_prev, f, f) = W.shape
    
    # Retrieve dimensions from dZ's shape
    (m, n_C, n_W, n_H) = dZ.shape
    
    # Initialize dA_prev, dW, db with the correct shapes
    dA_prev = torch.zeros((m, n_C_prev, n_W_prev, n_H_prev))                           
    dW = torch.zeros((n_C, n_C_prev, f, f))
    db = torch.zeros((n_C, 1, 1, 1))

    # Pad A_prev and dA_prev
    if padding is not None: 
        A_prev = padding(A_prev)
        dA_prev = padding(dA_prev)
        
    for h in range(n_H): # loop over vertical axis of the output volume
        for w in range(n_W):               # loop over horizontal axis of the output volume
                
                # Find the corners of the current "slice"
                vert_start = h*stride
                vert_end = vert_start + f
                horiz_start = w*stride
                horiz_end = horiz_start + f
                    
                # Use the corners to define the slice from a_prev_pad
                a_slice = A_prev[:, :, horiz_start:horiz_end, vert_start:vert_end]
                
                ezdz = dZ[:, :, w, h].unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)

                # Update gradients for the filter, bias and input
                dA_prev[:, :, horiz_start:horiz_end, vert_start:vert_end] += (W * ezdz).sum(dim=1)
                dW += (a_slice.unsqueeze(1)*ezdz).sum(dim=0)
                db += dZ[:, :, w, h].sum(dim=0).unsqueeze(-1).unsqueeze(-1).unsqueeze(-1)
                
    if padding is not None: dA_prev = dA_prev[:, :, padding.size:-padding.size, padding.size:-padding.size]
    
    weight.update(dW)
    bias.update(db.view(-1))
    inp.g = dA_prev
    ```

## Training

In [ ]:
#export

class Conv(Module):
    "Module to perform convolutions. Can specify kernel size, stride and padding"
    def __init__(self, n_in, n_out, kernel_size=3, stride=1, leak=1, padding=None):
        super().__init__()
        self.n_in, self.n_out = n_in, n_out
        self.filters = Parameter(get_weight(n_out, n_in, kernel_size, kernel_size, leak, True))
        self.bias = Parameter(torch.zeros(n_out))
        self.stride = stride
        self.kernel_size = kernel_size
        self.padding = padding
    
    def forward(self, xb): return convolve(xb, self.filters.d, self.bias.d, self.stride, self.padding)
    
    def bwd(self, out, inp): conv_back(out, inp, self.filters,self.bias,self.stride,self.padding)
        
    def __repr__(self): return f'Conv({self.n_in}, {self.n_out}, ks = {self.kernel_size}, stride = {self.stride})'

In [ ]:
def get_basic_conv_model(lr):
    "Helper function to get a basic conv model up and running"
    pad1 = Padding(1)
    pad2 = Padding(1)
    model = SequentialModel(Reshape(1, 28, 28), 
                            Conv(1, 16, 5, stride=3, leak=0, padding=pad1), 
                            ReLU(), 
                            Conv(16, 8, 5, stride=2, leak=1, padding=pad2), 
                            Flatten(), 
                            Linear(128, 10, False))
    loss_func = CrossEntropy()
    optimizer = Optimizer(model.parameters(), lr)
    return model, optimizer, loss_func

Testing vs Pytorch

In [ ]:
m, o, l = get_basic_conv_model(0.1)
loss_func = l
xt, yt, _, _ = get_mnist()
sx, sy = xt[:100], yt[:100]

In [ ]:
m

(Layer1): Reshape(1, 28, 28)
(Layer2): Conv(1, 16, ks = 5, stride = 3)
(Layer3): ReLU()
(Layer4): Conv(16, 8, ks = 5, stride = 2)
(Layer5): Flatten()
(Layer6): Linear(128, 10)

In [ ]:
loss = loss_func(m(sx), sy)
loss_func.backward()
m.backward()

In [ ]:
sxg = sx.g.clone()
cw1g = m.layers[1].filters.grad.clone()
cb1g = m.layers[1].bias.grad.clone()
cw2g = m.layers[3].filters.grad.clone()
cb2g = m.layers[3].bias.grad.clone()
lw = m.layers[5].w.grad.clone()
lb = m.layers[5].b.grad.clone()

In [ ]:
sx2 = sx.clone().requires_grad_(True)
m.layers[1].filters.d.requires_grad_(True)
m.layers[1].bias.d.requires_grad_(True)
m.layers[3].filters.d.requires_grad_(True)
m.layers[3].bias.d.requires_grad_(True)
m.layers[5].w.d.requires_grad_(True)
m.layers[5].b.d.requires_grad_(True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [ ]:
loss = loss_func(m(sx2), sy)
loss.backward()

In [ ]:
test_near(lb, m.layers[5].b.d.grad)
test_near(lw, m.layers[5].w.d.grad)
test_near(cb2g, m.layers[3].bias.d.grad)
test_near(cw2g, m.layers[3].filters.d.grad)
test_near(cb1g, m.layers[1].bias.d.grad)
test_near(cw1g, m.layers[1].filters.d.grad)

good
good
good
good
good
good


In [ ]:
#export
#hide

def get_linear_model(lr):
    model = SequentialModel(Linear(784, 50, True), ReLU(), Linear(50, 10, False))
    loss_func = CrossEntropy()
    optimizer = Optimizer(model.parameters(), lr)
    return model, optimizer, loss_func

def get_model(lr, modules):
    model = SequentialModel(*modules)
    loss_func = CrossEntropy()
    optimizer = Optimizer(model.parameters(), lr)
    return model, optimizer, loss_func

In [ ]:
m = [Reshape(1, 28, 28), 
     Conv(1, 4, 5, stride=1, leak=0), 
     ReLU(), 
     Conv(4, 1, 5, stride=1, leak=1), 
     Flatten(), 
     Linear(20*20, 10, False)]

m, o, l = get_model(0.1, m)

In [ ]:
#export
def get_small_datasets():
    "Helper function to get smaller versions of MNIST datasets"
    xt, yt, xv, yv = get_mnist()
    tr = Dataset(xt[:500], yt[:500])
    val = Dataset(xv[:100], yv[:100])
    train = DataLoader(tr, Batcher(tr, 64, True), collate)
    valid = DataLoader(val, Batcher(val, 64, False), collate)
    return train, valid

In [ ]:
train,valid = get_small_datasets()

In [ ]:
fit(2, m, o, l, train, valid)

Epoch 1, Accuracy: 0.0963541716337204, Loss: nan
Epoch 2, Accuracy: 0.0963541716337204, Loss: nan


# Pooling Layers

## Forward

In [ ]:
#export
def max_pool(inp): 
    "Applies a max pooling operation on `inp`"
    return inp.max(dim=-1).values.max(dim=-1).values

def avg_pool(inp): 
    "Applies an average pooling operation on `inp`"
    return torch.mean(inp,dim=(2,3))

def pool(inp, ks, stride, padding=None, operation=max_pool):
    "Runs a pooling operation on `inp` of type `operation` with given `stride`, `ks` and `padding`"
    if padding is not None: 
        if operation == max_pool: padding.value = inp.min() - 1
        inpp = padding(inp)
    else: inpp = inp
        
    bs, nc, h, w = inp.shape
    nw, nh = int((int(w - ks) / stride)+1), int((int(h - ks) / stride)+1)
    
    out = torch.zeros(bs, nc, nw, nh)
    
    for i in range(nh):
        for j in range(nw):
            window = inpp[:,:,j*stride:j*stride+ks, i*stride:i*stride+ks]
            out[:,:,j,i] = operation(window)
    
    return out

```python
def pool(inp, ks, stride, padding=None, operation=max_pool):
    if padding is not None: 
        if operation == max_pool: padding.value = inp.min() - 1
        inpp = padding(inp)
    else: inpp = inp
        
    bs, nc, h, w = inp.shape
    nw, nh = int((int(w - ks) / stride)+1), int((int(h - ks) / stride)+1)
    
    out = torch.zeros(bs, nc, nw, nh)
    
    for i in range(nh):
        for j in range(nw):
            window = inpp[:,:,j*stride:j*stride+ks, i*stride:i*stride+ks]
            out[:,:,j,i] = operation(window)
    
    return out
```

Testing Pooling Forward Passes

In [ ]:
test_t = torch.randn(16, 3, 28, 28)
ks = 3
stride = 1
pad_size = 0

In [ ]:
py_max = nn.MaxPool2d(ks, stride=stride, padding=pad_size)
py_avg = nn.AvgPool2d(ks, stride=stride, padding=pad_size)
py_max_result = py_max(test_t)
py_avg_result = py_avg(test_t)

In [ ]:
pad = Padding(size=pad_size) if pad_size > 0 else None
my_avg = pool(test_t, ks, stride, pad, avg_pool)
my_max = pool(test_t, ks, stride, pad, max_pool)

In [ ]:
test_near(py_max_result, my_max)
test_near(py_avg_result, my_avg)

good
good


## Backward

In [ ]:
#export
def max_back(window): 
    "Gradient for `window` of max pooling"
    A = torch.zeros(window.shape)
    bs, nc, _, _ = window.shape
    
    for i in range(bs):
        for j in range(nc):
            A[i,j,:,:] = window[i,j,:,:] == window[i,j,:,:].max()
    
    return A
    
    
def average_back(window, shape):
    "Gradient for `window` of average pooling"
    height, width = shape
    window_sum = window.unsqueeze(-1).unsqueeze(-1) / height*width
    return torch.ones(shape).unsqueeze(0).unsqueeze(0) * window_sum

def pool_back(out, inp, ks, stride, padding=None, operation=max_pool):
    "Function to pass gradient through pooling layers"
    dZ = out.g
    
    if padding is not None: 
        if operation == max_pool: padding.value = inp.min() - 1
        inp = padding(inp)
        
    bs, nc, nh, nw = dZ.shape

    dA_prev = torch.zeros(inp.shape)
    
    shape = (ks,ks)
    
    for i in range(nh):
        for j in range(nw):
            
            if operation == max_pool:
                window = inp[:,:,j*stride:j*stride+ks, i*stride:i*stride+ks]
                
                mask = max_back(window)
                
                dA_prev[:,:,j*stride:j*stride+ks, i*stride:i*stride+ks] += mask*dZ[:,:,j,i].unsqueeze(-1).unsqueeze(-1)
            
            elif operation == avg_pool:
                dz = dZ[:,:,j, i]
                dA_prev[:,:,j*stride:j*stride+ks, i*stride:i*stride+ks] += average_back(dz, shape)
    
    inp.g = dA_prev

```python
def pool_back(out, inp, ks, stride, padding=None, operation=max_pool):
    dZ = out.g
    
    if padding is not None: 
        if operation == max_pool: padding.value = inp.min() - 1
        inp = padding(inp)
        
    bs, nc, nh, nw = dZ.shape

    dA_prev = torch.zeros(inp.shape)
    
    shape = (ks,ks)
    
    for i in range(nh):
        for j in range(nw):
            
            if operation == max_pool:
                window = inp[:,:,j*stride:j*stride+ks, i*stride:i*stride+ks]
                
                mask = max_back(window)
                
                dA_prev[:,:,j*stride:j*stride+ks, i*stride:i*stride+ks] += mask*dZ[:,:,j,i].unsqueeze(-1).unsqueeze(-1)
            
            elif operation == avg_pool:
                dz = dZ[:,:,j, i]
                dA_prev[:,:,j*stride:j*stride+ks, i*stride:i*stride+ks] += average_back(dz, shape)
    
    inp.g = dA_prev
    ```

## Module

In [ ]:
#export
class Pool(Module):
    "Module for defining a pooling layer in a model"
    def __init__(self, operation, ks=2, stride=2, padding=None):
        super().__init__()
        self.ks, self.stride, self.padding, self.operation = ks, stride, padding, operation
    
    def forward(self, xb): return pool(xb, self.ks, self.stride, self.padding, operation=self.operation)
    
    def bwd(self, out, inp): return pool_back(out, inp, self.ks, self.stride, self.padding, operation=self.operation)
    
    def __repr__(self): 
        if self.operation == max_pool: ptype = "Max"
        elif self.operation == avg_pool: ptype = "Average"
        else: ptype = "Custom"
        return f'{ptype}Pool(ks: {self.ks}, stride: {self.stride})'

In [ ]:
layers = [Reshape(1, 28, 28),
          Conv(1, 4),
          Pool(avg_pool, ks=2, stride=1),
          Conv(4, 1),
          Flatten(),
          Linear(529, 10, False)]
m, o, l = get_model(0.1, layers)
train,valid = get_small_datasets()

In [ ]:
m

(Layer1): Reshape(1, 28, 28)
(Layer2): Conv(1, 4, ks = 3, stride = 1)
(Layer3): AveragePool(ks: 2, stride: 1)
(Layer4): Conv(4, 1, ks = 3, stride = 1)
(Layer5): Flatten()
(Layer6): Linear(529, 10)

In [ ]:
loss = loss_func(m(sx), sy)
loss_func.backward()
m.backward()

In [ ]:
sxg = sx.g.clone()
cw = m.layers[1].filters.grad.clone()
cb = m.layers[1].bias.grad.clone()
lw = m.layers[5].w.grad.clone()
lb = m.layers[5].b.grad.clone()

In [ ]:
sx2 = sx.clone().requires_grad_(True)
m.layers[1].filters.d.requires_grad_(True)
m.layers[1].bias.d.requires_grad_(True)
m.layers[5].w.d.requires_grad_(True)
m.layers[5].b.d.requires_grad_(True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)